# Prompt Templates

Now that I have my vector database setup, I want to pull down the most likely schema and table and save the metadata as variables. Then I think the best way to proceed to is to create a prompt template that accepts the variables and allows a consistent entry point to the model.

## Imports

In [7]:
import pandas as pd

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


## Load Top Results from VectorDB

In [2]:
#setup embeddings using HuggingFace and the directory location
embeddings  = HuggingFaceEmbeddings()
persist_dir = '../data/processed/chromadb/schema-table-split'

# load from disk
vectordb = Chroma(persist_directory=persist_dir, embedding_function=embeddings)

In [3]:
#run prompt as query and get most likely results
query = "How many heads of the departments are older than 56?" #first prompt from the training data

result = vectordb.similarity_search(query, k=1)

In [4]:
result

[Document(page_content='Schema Table: department management head', metadata={'schema': 'department_management', 'table': 'head', 'columns': '["head_id", "name", "born_state", "age"]'})]

## Save Metadata to Variables

In [6]:
top_schema = result[0].metadata['schema']
top_table = result[0].metadata['table']
table_cols = result[0].metadata['columns']

print(top_schema, top_table, table_cols)

department_management head ["head_id", "name", "born_state", "age"]


## Create SQL Database Chain

Langchain has a great SQL Agent we can tap into. It allows for connecting to huggingface models, although it warns about not being able to run complicated tasks due to the lack of processing power. For the simplest test, what we'll need to do is point to the database (in this case the schema returned by our vector query), establish which LLM model we're using, and run the prompt on the chain. We may even use the option to specify the table to give it very little thinking to do.

There is also more advanced features we'll tap into down the road, like Sequential Chaining that takes the database and then links tables together to find the right answer and Query Checker that can solve for some issues in the query. But we'll get to those later.

### SQL Chain Imports

In [8]:
from langchain import SQLDatabase, SQLDatabaseChain
import logging
import torch
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from langchain import HuggingFacePipeline

### Establish LLM

In [ ]:
model_id = "tiiuae/falcon-7b-instruct"

device_id = -1  # default to no-GPU, but use GPU and half precision mode if available
if torch.cuda.is_available():
    device_id = 0
    try:
        model = model_id.half()
    except RuntimeError as exc:
        logging.warn(f"Could not run model in half precision mode: {str(exc)}")

tokenizer = AutoTokenizer.from_pretrained(model_id)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024, device=device_id, trust_remote_code=True)

local_llm = HuggingFacePipeline(pipeline=pipe)

This appears to be working when I start it up, but I haven't ran it all the way through since I don't fully understand all the possibilities when it comes to using a huggingface model. Is there a way to access private non-OpenAI models?